# Import thư viện cần thiết

In [1]:
import json
import pandas as pd
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import learning_curve


In [2]:
import numpy as np
import pandas as pd
import os
from scipy import sparse
from scipy.sparse import csr_matrix, save_npz, load_npz

# Load file json

In [5]:
def load_json_file(path):
    with open(path, 'r', encoding='utf-8') as f:
        raw = f.read().strip()
        try:
            data = json.loads(raw)
            if isinstance(data, dict):
                data = [data]
        except json.JSONDecodeError:
            data = [json.loads(line) for line in raw.splitlines() if line.strip()]
    return pd.DataFrame(data)

PATH = "/Users/anhquan/Workspace/CTAI_MachineLearning_Project/data/processed_data/processed_data.json"
df = load_json_file(PATH)


In [6]:
df.head()

,url,url_img,title,description,content,metadata,title_clean,desc_clean,content_clean
0,https://vietnamnet.vn/tre-21-thang-tuoi-phai-c...,https://static-images.vnncdn.net/vps_images_pu...,Trẻ 21 tháng tuổi phải cấp cứu sau khi con rắn...,Trẻ 21 tháng tuổi được đưa vào cấp cứu trong t...,"Mới đây, Bệnh viện Sản Nhi Quảng Ninh tiếp nhậ...","{'cat': 'sức khỏe', 'subcat': 'Tin tức', 'publ...",trẻ 21 cấp_cứu rắn đen - trắng bò cạnh,trẻ 21 cấp_cứu tình_trạng tím_tái li_bì phát_h...,mới_đây bệnh_viện sản_nhi quảng_ninh tiếp_nhận...
1,https://vietnamnet.vn/lieu-thuoc-vo-gia-giup-n...,https://static-images.vnncdn.net/vps_images_pu...,'Liều thuốc vô giá' giúp người phụ nữ vượt qua...,"Trong 3 năm, bà Thuận phát hiện mắc 3 bệnh ung...",3 năm phát hiện 3 bệnh ung thư\nBà Phan Thanh ...,"{'cat': 'sức khỏe', 'subcat': 'Tin tức', 'publ...",liều thuốc vô_giá giúp phụ_nữ 3 căn_bệnh ung_thư,3 thuận phát_hiện mắc 3 bệnh ung_thư 3 bàn mổ ...,3 phát_hiện 3 bệnh ung_thư phan_thanh_thuận 68...
2,https://vietnamnet.vn/su-hoi-sinh-cua-chang-ca...,https://static-images.vnncdn.net/vps_images_pu...,Sự hồi sinh của chàng ca sĩ trẻ từng chìm sâu ...,"Nhìn tôi đứng trên sân khấu, biểu diễn ca khúc...","Nguyễn Thế Dũng sinh năm 2002, quê Hoà Bình, h...","{'cat': 'sức khỏe', 'subcat': 'Tin tức', 'publ...",hồi_sinh chàng ca_sĩ trẻ chìm sâu giấc_ngủ đôn...,đứng sân_khấu biểu_diễn ca_khúc sáng_tác ngày_...,nguyễn_thế dũng_sinh 2002 quê hòa bình hiện ca...
3,https://vietnamnet.vn/tung-xin-duoc-chet-chang...,https://static-images.vnncdn.net/vps_images_pu...,"Từng xin được chết, chàng trai hồi sinh kỳ diệ...",Đa phần những bệnh nhân bị tai nạn như Thịnh c...,"""Mẹ cho con về đi, chết ở nhà cũng được""\nThán...","{'cat': 'sức khỏe', 'subcat': 'Tin tức', 'publ...",chết chàng trai hồi_sinh kỳ_diệu bác_sĩ bất_ngờ,đa_phần bệnh_nhân tai_nạn thịnh nguy_cơ tử_von...,mẹ đi chết 6 2021 nguyễn_văn_thịnh sinh 1994 t...
4,https://vietnamnet.vn/tuoi-tho-nguoi-viet-tang...,https://static-images.vnncdn.net/vps_images_pu...,"Tuổi thọ người Việt tăng mạnh, nơi nào người d...","Tuổi thọ người Việt Nam năm 2023 tăng lên 74,5...",Tổng cục Thống kê vừa công bố Niên giám thống ...,"{'cat': 'sức khỏe', 'subcat': 'Tin tức', 'publ...",tuổi_thọ việt người_dân sống,tuổi_thọ việt_nam 2023 74 5 - 1 trung_bình 4 n...,tổng_cục thống_kê công_bố_niên giám_thống_kê 2...


In [7]:
meta_df = pd.json_normalize(df['metadata'])
meta_df.head(20)

,cat,subcat,published_date,author
0,sức khỏe,Tin tức,20/08/2025 - 09:30,Phương Thuý
1,sức khỏe,Tin tức,11/09/2024 - 05:38,Unknown
2,sức khỏe,Tin tức,23/08/2024 - 08:25,Võ Thu
3,sức khỏe,Tin tức,16/08/2024 - 05:38,Phương Thuý
4,sức khỏe,Tin tức,23/07/2024 - 10:13,Unknown
5,sức khỏe,Tin tức,24/07/2024 - 08:49,Võ Thu
6,sức khỏe,Tin tức,25/07/2024 - 08:43,Võ Thu
7,sức khỏe,Tin tức,07/09/2024 - 16:38,Võ Thu
8,sức khỏe,Tin tức,05/09/2024 - 10:41,Võ Thu
9,sức khỏe,Tin tức,04/09/2024 - 10:36,Võ Thu


In [8]:
col = meta_df['subcat']

cnt_none = (col.apply(lambda x: x is None)).sum()
cnt_nan  = col.isna().sum()
cnt_empty = (col.astype(str).str.strip() == "").sum()   

print("None (is None):", cnt_none)
print("NaN (isna):    ", cnt_nan)
print('Empty "":      ', cnt_empty)

total_blank = ((col.isna()) | (col.astype(str).str.strip() == "")).sum()
print("Total blank:", total_blank)


None (is None): 318
NaN (isna):     318
Empty "":       3753
Total blank: 4071


In [9]:
meta_df['subcat'] = meta_df['subcat'].replace({
    "": np.nan,
    " ": np.nan,
    "nan": np.nan,
    "None": np.nan
})

# Nếu có giá trị None của Python thì pandas.isna() sẽ bắt được
meta_df['subcat'] = meta_df['subcat'].where(~meta_df['subcat'].isna(), np.nan)

# Kiểm tra lại
print(meta_df['subcat'].value_counts(dropna=False))
print("Tổng số NaN:", meta_df['subcat'].isna().sum())


subcat
NaN                     4882
Tin tức                  393
Tuyển sinh               161
Quân sự                   94
Thị trường                87
                        ... 
Việt Nam và thế giới       1
Coaching                   1
Cộng đồng                  1
Bảo hiểm                   1
Tâm sự                     1
Name: count, Length: 130, dtype: int64
Tổng số NaN: 4882


In [10]:
meta_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8258 entries, 0 to 8257
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   cat             8258 non-null   object
 1   subcat          3376 non-null   object
 2   published_date  8258 non-null   object
 3   author          8258 non-null   object
dtypes: object(4)
memory usage: 258.2+ KB


In [11]:
df_merge = pd.concat([df.drop(columns=['metadata']), meta_df], axis=1)

In [12]:
df_merge.head()

,url,url_img,title,description,content,title_clean,desc_clean,content_clean,cat,subcat,published_date,author
0,https://vietnamnet.vn/tre-21-thang-tuoi-phai-c...,https://static-images.vnncdn.net/vps_images_pu...,Trẻ 21 tháng tuổi phải cấp cứu sau khi con rắn...,Trẻ 21 tháng tuổi được đưa vào cấp cứu trong t...,"Mới đây, Bệnh viện Sản Nhi Quảng Ninh tiếp nhậ...",trẻ 21 cấp_cứu rắn đen - trắng bò cạnh,trẻ 21 cấp_cứu tình_trạng tím_tái li_bì phát_h...,mới_đây bệnh_viện sản_nhi quảng_ninh tiếp_nhận...,sức khỏe,Tin tức,20/08/2025 - 09:30,Phương Thuý
1,https://vietnamnet.vn/lieu-thuoc-vo-gia-giup-n...,https://static-images.vnncdn.net/vps_images_pu...,'Liều thuốc vô giá' giúp người phụ nữ vượt qua...,"Trong 3 năm, bà Thuận phát hiện mắc 3 bệnh ung...",3 năm phát hiện 3 bệnh ung thư\nBà Phan Thanh ...,liều thuốc vô_giá giúp phụ_nữ 3 căn_bệnh ung_thư,3 thuận phát_hiện mắc 3 bệnh ung_thư 3 bàn mổ ...,3 phát_hiện 3 bệnh ung_thư phan_thanh_thuận 68...,sức khỏe,Tin tức,11/09/2024 - 05:38,Unknown
2,https://vietnamnet.vn/su-hoi-sinh-cua-chang-ca...,https://static-images.vnncdn.net/vps_images_pu...,Sự hồi sinh của chàng ca sĩ trẻ từng chìm sâu ...,"Nhìn tôi đứng trên sân khấu, biểu diễn ca khúc...","Nguyễn Thế Dũng sinh năm 2002, quê Hoà Bình, h...",hồi_sinh chàng ca_sĩ trẻ chìm sâu giấc_ngủ đôn...,đứng sân_khấu biểu_diễn ca_khúc sáng_tác ngày_...,nguyễn_thế dũng_sinh 2002 quê hòa bình hiện ca...,sức khỏe,Tin tức,23/08/2024 - 08:25,Võ Thu
3,https://vietnamnet.vn/tung-xin-duoc-chet-chang...,https://static-images.vnncdn.net/vps_images_pu...,"Từng xin được chết, chàng trai hồi sinh kỳ diệ...",Đa phần những bệnh nhân bị tai nạn như Thịnh c...,"""Mẹ cho con về đi, chết ở nhà cũng được""\nThán...",chết chàng trai hồi_sinh kỳ_diệu bác_sĩ bất_ngờ,đa_phần bệnh_nhân tai_nạn thịnh nguy_cơ tử_von...,mẹ đi chết 6 2021 nguyễn_văn_thịnh sinh 1994 t...,sức khỏe,Tin tức,16/08/2024 - 05:38,Phương Thuý
4,https://vietnamnet.vn/tuoi-tho-nguoi-viet-tang...,https://static-images.vnncdn.net/vps_images_pu...,"Tuổi thọ người Việt tăng mạnh, nơi nào người d...","Tuổi thọ người Việt Nam năm 2023 tăng lên 74,5...",Tổng cục Thống kê vừa công bố Niên giám thống ...,tuổi_thọ việt người_dân sống,tuổi_thọ việt_nam 2023 74 5 - 1 trung_bình 4 n...,tổng_cục thống_kê công_bố_niên giám_thống_kê 2...,sức khỏe,Tin tức,23/07/2024 - 10:13,Unknown


In [13]:
print(df_merge['cat'].unique())
df_merge['cat'].value_counts()


['sức khỏe' 'thể thao' 'văn hóa - giải trí' 'xã hội' 'du lịch' 'chính trị'
 'pháp luật' 'thế giới' 'khoa học - công nghệ' 'giáo dục' 'kinh tế']


cat
xã hội                  874
sức khỏe                854
văn hóa - giải trí      801
thể thao                775
kinh tế                 765
giáo dục                756
thế giới                752
du lịch                 733
pháp luật               695
khoa học - công nghệ    652
chính trị               601
Name: count, dtype: int64

In [14]:
df_merge.head()

,url,url_img,title,description,content,title_clean,desc_clean,content_clean,cat,subcat,published_date,author
0,https://vietnamnet.vn/tre-21-thang-tuoi-phai-c...,https://static-images.vnncdn.net/vps_images_pu...,Trẻ 21 tháng tuổi phải cấp cứu sau khi con rắn...,Trẻ 21 tháng tuổi được đưa vào cấp cứu trong t...,"Mới đây, Bệnh viện Sản Nhi Quảng Ninh tiếp nhậ...",trẻ 21 cấp_cứu rắn đen - trắng bò cạnh,trẻ 21 cấp_cứu tình_trạng tím_tái li_bì phát_h...,mới_đây bệnh_viện sản_nhi quảng_ninh tiếp_nhận...,sức khỏe,Tin tức,20/08/2025 - 09:30,Phương Thuý
1,https://vietnamnet.vn/lieu-thuoc-vo-gia-giup-n...,https://static-images.vnncdn.net/vps_images_pu...,'Liều thuốc vô giá' giúp người phụ nữ vượt qua...,"Trong 3 năm, bà Thuận phát hiện mắc 3 bệnh ung...",3 năm phát hiện 3 bệnh ung thư\nBà Phan Thanh ...,liều thuốc vô_giá giúp phụ_nữ 3 căn_bệnh ung_thư,3 thuận phát_hiện mắc 3 bệnh ung_thư 3 bàn mổ ...,3 phát_hiện 3 bệnh ung_thư phan_thanh_thuận 68...,sức khỏe,Tin tức,11/09/2024 - 05:38,Unknown
2,https://vietnamnet.vn/su-hoi-sinh-cua-chang-ca...,https://static-images.vnncdn.net/vps_images_pu...,Sự hồi sinh của chàng ca sĩ trẻ từng chìm sâu ...,"Nhìn tôi đứng trên sân khấu, biểu diễn ca khúc...","Nguyễn Thế Dũng sinh năm 2002, quê Hoà Bình, h...",hồi_sinh chàng ca_sĩ trẻ chìm sâu giấc_ngủ đôn...,đứng sân_khấu biểu_diễn ca_khúc sáng_tác ngày_...,nguyễn_thế dũng_sinh 2002 quê hòa bình hiện ca...,sức khỏe,Tin tức,23/08/2024 - 08:25,Võ Thu
3,https://vietnamnet.vn/tung-xin-duoc-chet-chang...,https://static-images.vnncdn.net/vps_images_pu...,"Từng xin được chết, chàng trai hồi sinh kỳ diệ...",Đa phần những bệnh nhân bị tai nạn như Thịnh c...,"""Mẹ cho con về đi, chết ở nhà cũng được""\nThán...",chết chàng trai hồi_sinh kỳ_diệu bác_sĩ bất_ngờ,đa_phần bệnh_nhân tai_nạn thịnh nguy_cơ tử_von...,mẹ đi chết 6 2021 nguyễn_văn_thịnh sinh 1994 t...,sức khỏe,Tin tức,16/08/2024 - 05:38,Phương Thuý
4,https://vietnamnet.vn/tuoi-tho-nguoi-viet-tang...,https://static-images.vnncdn.net/vps_images_pu...,"Tuổi thọ người Việt tăng mạnh, nơi nào người d...","Tuổi thọ người Việt Nam năm 2023 tăng lên 74,5...",Tổng cục Thống kê vừa công bố Niên giám thống ...,tuổi_thọ việt người_dân sống,tuổi_thọ việt_nam 2023 74 5 - 1 trung_bình 4 n...,tổng_cục thống_kê công_bố_niên giám_thống_kê 2...,sức khỏe,Tin tức,23/07/2024 - 10:13,Unknown


In [15]:
df_new = df_merge[['title_clean', 'desc_clean', 'content_clean', 'cat']].copy()
df_new.head()

,title_clean,desc_clean,content_clean,cat
0,trẻ 21 cấp_cứu rắn đen - trắng bò cạnh,trẻ 21 cấp_cứu tình_trạng tím_tái li_bì phát_h...,mới_đây bệnh_viện sản_nhi quảng_ninh tiếp_nhận...,sức khỏe
1,liều thuốc vô_giá giúp phụ_nữ 3 căn_bệnh ung_thư,3 thuận phát_hiện mắc 3 bệnh ung_thư 3 bàn mổ ...,3 phát_hiện 3 bệnh ung_thư phan_thanh_thuận 68...,sức khỏe
2,hồi_sinh chàng ca_sĩ trẻ chìm sâu giấc_ngủ đôn...,đứng sân_khấu biểu_diễn ca_khúc sáng_tác ngày_...,nguyễn_thế dũng_sinh 2002 quê hòa bình hiện ca...,sức khỏe
3,chết chàng trai hồi_sinh kỳ_diệu bác_sĩ bất_ngờ,đa_phần bệnh_nhân tai_nạn thịnh nguy_cơ tử_von...,mẹ đi chết 6 2021 nguyễn_văn_thịnh sinh 1994 t...,sức khỏe
4,tuổi_thọ việt người_dân sống,tuổi_thọ việt_nam 2023 74 5 - 1 trung_bình 4 n...,tổng_cục thống_kê công_bố_niên giám_thống_kê 2...,sức khỏe


# Xử lý dữ liệu TF-IDF

In [17]:
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

In [22]:
df_new["text"] = (df_new["title_clean"] + " " +
                  df_new["desc_clean"]  + " " +
                  df_new["content_clean"]).astype(str)

X = df_new["text"]
le = LabelEncoder()
y = le.fit_transform(df_new["cat"])
# y = df_new["cat"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=42
)

vectorizer = TfidfVectorizer(
    max_features=50000,
    min_df=2,
    max_df=0.85,
    ngram_range=(1, 3),
    lowercase=True
)

Xtr = vectorizer.fit_transform(X_train)
Xte = vectorizer.transform(X_test)

print("Train shape:", Xtr.shape, " Test shape:", Xte.shape)
print("10 feature đầu tiên:", vectorizer.get_feature_names_out()[:10])


Train shape: (6193, 50000)  Test shape: (2065, 50000)
10 feature đầu tiên: ['00' '000' '000 000' '000 000 đồng' '000 30' '000 50' '000 500'
 '000 baht' '000 bước' '000 bảng']


# Training

## Linear SVM

In [23]:
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

svm_clf = LinearSVC(class_weight='balanced', random_state=42)
svm_clf.fit(Xtr, y_train)
svm_pred = svm_clf.predict(Xte)
svm_pred_train = svm_clf.predict(Xtr)

print(classification_report(y_test, svm_pred, digits=4))
print("Confusion matrix:\n", confusion_matrix(y_test, svm_pred))
print("Accuracy:", accuracy_score(y_test, svm_pred))
print("Accuracy train:", accuracy_score(y_train, svm_pred_train))
print("F1-macro:", f1_score(y_test, svm_pred, average='macro'))
print("F1-weighted:", f1_score(y_test, svm_pred, average='weighted'))

              precision    recall  f1-score   support

           0     0.7194    0.6667    0.6920       150
           1     0.8614    0.9508    0.9039       183
           2     0.9424    0.9524    0.9474       189
           3     0.8688    0.8528    0.8607       163
           4     0.8513    0.8691    0.8601       191
           5     0.8280    0.8851    0.8556       174
           6     0.9283    0.9673    0.9474       214
           7     0.9653    0.8883    0.9252       188
           8     0.9797    0.9948    0.9872       194
           9     0.9167    0.9350    0.9257       200
          10     0.6974    0.6210    0.6570       219

    accuracy                         0.8731      2065
   macro avg     0.8690    0.8712    0.8693      2065
weighted avg     0.8708    0.8731    0.8712      2065

Confusion matrix:
 [[100   3   0   3   1  22   1   1   0   3  16]
 [  1 174   0   0   0   0   0   0   1   4   3]
 [  0   0 180   1   2   1   0   0   1   1   3]
 [  0   5   2 139   6   0  

In [24]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

# KFold giữ tỉ lệ lớp
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Model + không gian tham số
base = LinearSVC(class_weight="balanced", random_state=42)
param_grid = {
    "C": [0.05, 0.1, 0.2, 0.5, 1, 2, 5, 10],
    "loss": ["hinge", "squared_hinge"],   # thử 2 loss
    # Với dữ liệu TF-IDF thường #features >> #samples -> dual=True là hợp lý
    "dual": [True]
}

grid = GridSearchCV(
    estimator=base,
    param_grid=param_grid,
    scoring={"f1_macro": "f1_macro", "accuracy": "accuracy"},
    refit="f1_macro",
    cv=cv,
    n_jobs=-1,
    verbose=1
)

print("- Tuning LinearSVC với GridSearchCV…")
grid.fit(Xtr, y_train)


print("Best params:", grid.best_params_)
print("Best CV f1_macro:", grid.best_score_)

# Đánh giá trên test
svm_best = grid.best_estimator_
svm_pred = svm_best.predict(Xte)
svm_pred_train = svm_best.predict(Xtr)

print("\nClassification report (test):")
print(classification_report(y_test, svm_pred, digits=4))
print("Confusion matrix:\n", confusion_matrix(y_test, svm_pred))
print("Accuracy:", accuracy_score(y_test, svm_pred))
print("Accuracy train:", accuracy_score(y_train, svm_pred_train))
print("F1-macro:", f1_score(y_test, svm_pred, average="macro"))
print("F1-weighted:", f1_score(y_test, svm_pred, average="weighted"))


- Tuning LinearSVC với GridSearchCV…
Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best params: {'C': 0.5, 'dual': True, 'loss': 'squared_hinge'}
Best CV f1_macro: 0.8509444058539761

Classification report (test):
              precision    recall  f1-score   support

           0     0.7000    0.6533    0.6759       150
           1     0.8621    0.9563    0.9067       183
           2     0.9368    0.9418    0.9393       189
           3     0.8797    0.8528    0.8660       163
           4     0.8376    0.8639    0.8505       191
           5     0.8000    0.8966    0.8455       174
           6     0.9238    0.9626    0.9428       214
           7     0.9649    0.8777    0.9192       188
           8     0.9797    0.9948    0.9872       194
           9     0.9122    0.9350    0.9235       200
          10     0.7043    0.5982    0.6469       219

    accuracy                         0.8683      2065
   macro avg     0.8637    0.8666    0.8640      2065
weighted avg 

## SVM

In [26]:
print("- Training ...")
print("- Train size = {}".format(X_train.shape))
model = svm.SVC(kernel='linear', C=1)
model.fit(Xtr, y_train)
print("- model - train complete")


- Training ...
- Train size = (6193,)
- model - train complete


In [27]:
from sklearn.metrics import accuracy_score
print("- Testing ...")
y_test_pred = model.predict(Xte)
y_train_pred = model.predict(Xtr)
print("- Test Acc = {}".format(accuracy_score(y_test, y_test_pred)))
print("- Train Acc = {}".format(accuracy_score(y_train, y_train_pred)))

- Testing ...
- Test Acc = 0.8576271186440678
- Train Acc = 0.971903762312288


# Naive Bayes

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import sklearn.naive_bayes as naive_bayes
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import learning_curve


In [29]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

print("- Training MNB ...")
print("- Train size = {}".format(Xtr.shape))

model_MNB = MultinomialNB(alpha=1.0)
model_MNB.fit(Xtr, y_train)

# Đánh giá nhanh
y_pred_train = model_MNB.predict(Xtr)
y_pred_test  = model_MNB.predict(Xte)

print("- Train acc = {:.4f}".format(accuracy_score(y_train, y_pred_train)))
print("- Test  acc = {:.4f}".format(accuracy_score(y_test,  y_pred_test)))
print()
print(classification_report(y_test, y_pred_test))

print("- model_MNB - train complete")

- Training MNB ...
- Train size = (6193, 50000)
- Train acc = 0.8682
- Test  acc = 0.8194

              precision    recall  f1-score   support

           0       0.67      0.51      0.58       150
           1       0.88      0.88      0.88       183
           2       0.95      0.90      0.92       189
           3       0.95      0.66      0.78       163
           4       0.80      0.90      0.85       191
           5       0.76      0.89      0.82       174
           6       0.89      0.94      0.91       214
           7       0.94      0.81      0.87       188
           8       0.98      0.97      0.98       194
           9       0.76      0.95      0.84       200
          10       0.52      0.53      0.52       219

    accuracy                           0.82      2065
   macro avg       0.83      0.81      0.81      2065
weighted avg       0.82      0.82      0.82      2065

- model_MNB - train complete


# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [31]:
knn = KNeighborsClassifier(
    n_neighbors=5,         # k mặc định
    metric="cosine",       # phù hợp TF-IDF
    algorithm="brute",     # để hỗ trợ cosine + sparse
    weights="distance",    # gần hơn -> ảnh hưởng mạnh hơn
    n_jobs=-1
)
knn.fit(Xtr, y_train)

y_pred = knn.predict(Xte)
y_pred_train = knn.predict(Xtr)

print("Accuracy train:", accuracy_score(y_train, y_pred_train))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1-macro:", f1_score(y_test, y_pred, average="macro"))
print("F1-weighted:", f1_score(y_test, y_pred, average="weighted"))
print("\nClassification report:")
print(classification_report(y_test, y_pred, target_names=le.classes_, digits=4))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy train: 1.0
Accuracy: 0.8111380145278451
F1-macro: 0.8060513887204013
F1-weighted: 0.809279835317313

Classification report:
                      precision    recall  f1-score   support

           chính trị     0.6040    0.6000    0.6020       150
             du lịch     0.8254    0.8525    0.8387       183
            giáo dục     0.9037    0.8942    0.8989       189
khoa học - công nghệ     0.7975    0.7730    0.7850       163
             kinh tế     0.7958    0.7958    0.7958       191
           pháp luật     0.7374    0.7586    0.7479       174
            sức khỏe     0.8918    0.9626    0.9258       214
            thế giới     0.8814    0.8298    0.8548       188
            thể thao     0.9453    0.9794    0.9620       194
  văn hóa - giải trí     0.8775    0.8950    0.8861       200
              xã hội     0.5980    0.5434    0.5694       219

            accuracy                         0.8111      2065
           macro avg     0.8052    0.8077    0.8061      20

# RF

In [32]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    max_features="sqrt",       # lấy sqrt(#features) mỗi node (giảm nhiễu)
    min_samples_leaf=1,
    class_weight="balanced",   # nếu lệch lớp
    n_jobs=-1,
    random_state=42
)
rf.fit(Xtr, y_train)

y_pred_tr = rf.predict(Xtr)
y_pred_te = rf.predict(Xte)

print("Train acc  :", accuracy_score(y_train, y_pred_tr))
print("Test  acc  :", accuracy_score(y_test,  y_pred_te))
print("Test F1-m  :", f1_score(y_test, y_pred_te, average="macro"))
print("Test F1-w  :", f1_score(y_test, y_pred_te, average="weighted"))
print("\nClassification report (test):")
print(classification_report(y_test, y_pred_te, target_names=le.classes_, digits=4))
print("Confusion matrix (test):\n", confusion_matrix(y_test, y_pred_te))


Train acc  : 1.0
Test  acc  : 0.8246973365617434
Test F1-m  : 0.8172134935153527
Test F1-w  : 0.8177740556235794

Classification report (test):
                      precision    recall  f1-score   support

           chính trị     0.6738    0.6333    0.6529       150
             du lịch     0.8358    0.9180    0.8750       183
            giáo dục     0.8832    0.9206    0.9016       189
khoa học - công nghệ     0.9174    0.6810    0.7817       163
             kinh tế     0.7895    0.8639    0.8250       191
           pháp luật     0.7364    0.9310    0.8223       174
            sức khỏe     0.8559    0.9439    0.8978       214
            thế giới     0.9341    0.8298    0.8789       188
            thể thao     0.9598    0.9845    0.9720       194
  văn hóa - giải trí     0.8178    0.9200    0.8659       200
              xã hội     0.6376    0.4338    0.5163       219

            accuracy                         0.8247      2065
           macro avg     0.8219    0.8236    0.8

# Stacking

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack, csr_matrix
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score




In [ ]:
df = df_new.copy()
df['text'] = df['title_clean'] + ' ' + df['desc_clean'] + ' ' + df['content_clean']
le = LabelEncoder()
y = le.fit_transform(df['cat'])

X_train, X_test, y_train, y_test = train_test_split(df['text'], y, test_size=0.25, random_state=42, stratify=y)


In [36]:
# Extract statistical features from text
def extract_text_statistics(df):
    """Extract statistical features from text"""

    # Basic text statistics
    df['text_length'] = df['text'].str.len()
    df['word_count'] = df['text'].str.split().str.len()
    df['avg_word_length'] = df['text'].apply(lambda x: np.mean([len(word) for word in x.split()]) if x.split() else 0)
    df['sentence_count'] = df['text'].str.count(r'[.!?]+')

    # Punctuation features (escape special regex characters)
    df['exclamation_count'] = df['text'].str.count(r'!')
    df['question_count'] = df['text'].str.count(r'\?')  # Escape the question mark
    df['comma_count'] = df['text'].str.count(r',')
    df['uppercase_ratio'] = df['text'].apply(lambda x: sum(1 for c in x if c.isupper()) / len(x) if len(x) > 0 else 0)

    # Special characters and numbers
    df['number_count'] = df['text'].str.count(r'\d')
    df['special_char_count'] = df['text'].str.count(r'[^\w\s]')

    return df

# Apply statistical features
print("📊 Extracting statistical features...")
df_with_stats = extract_text_statistics(df.copy())

# Show the new statistical features
stat_features = ['text_length', 'word_count', 'avg_word_length', 'sentence_count',
                'exclamation_count', 'question_count', 'comma_count', 'uppercase_ratio',
                'number_count', 'special_char_count']

print("Statistical features summary:")
print(df_with_stats[stat_features].describe())

📊 Extracting statistical features...
Statistical features summary:
        text_length   word_count  avg_word_length  sentence_count  \
count   8258.000000  8258.000000      8258.000000     8258.000000   
mean    2997.250303   489.361831         5.143892       27.090821   
std     1632.810557   263.688254         0.499867       13.666869   
min      701.000000   119.000000         3.535817        3.000000   
25%     1902.250000   310.000000         4.791096       18.000000   
50%     2690.000000   436.000000         5.103550       24.000000   
75%     3730.000000   608.000000         5.469369       33.000000   
max    25109.000000  3713.000000         7.063927      185.000000   

       exclamation_count  question_count  comma_count  uppercase_ratio  \
count             8258.0          8258.0       8258.0           8258.0   
mean                 0.0             0.0          0.0              0.0   
std                  0.0             0.0          0.0              0.0   
min            

In [35]:
tfidf_vectorizer = TfidfVectorizer(
    max_features=15000,  # Limit features for efficiency
    min_df=2,          # Ignore terms that appear in less than 2 documents
    max_df=0.85,       # Ignore terms that appear in more than 85% of documents
    ngram_range=(1, 3), # Use unigrams, bigrams, trigrams
    lowercase=True
)

X_train_text = X_train.reset_index(drop=True)
X_test_text = X_test.reset_index(drop=True)

Xtr = tfidf_vectorizer.fit_transform(X_train_text)
Xte = tfidf_vectorizer.transform(X_test_text)

print("Train shape:", Xtr.shape, " Test shape:", Xte.shape)

Train shape: (6193, 15000)  Test shape: (2065, 15000)


In [ ]:
train_indices = X_train.index
test_indices = X_test.index

X_train_stats = df_with_stats.loc[train_indices, stat_features]
X_test_stats = df_with_stats.loc[test_indices, stat_features]

scaler = StandardScaler()
X_train_stats_scaled = scaler.fit_transform(X_train_stats)
X_test_stats_scaled = scaler.transform(X_test_stats)

X_train_stats_sparse = csr_matrix(X_train_stats_scaled)
X_test_stats_sparse = csr_matrix(X_test_stats_scaled)

print("Stat features shape - Train:", X_train_stats_sparse.shape, " Test:", X_test_stats_sparse.shape)
print("Stastical features:", stat_features)

X_train_combined = sparse.hstack([Xtr, X_train_stats_sparse])
X_test_combined = sparse.hstack([Xte, X_test_stats_sparse])



print("Combined features shape - Train:", X_train_combined.shape, " Test:", X_test_combined.shape)

Stat features shape - Train: (6193, 10)  Test: (2065, 10)
Stastical features: ['text_length', 'word_count', 'avg_word_length', 'sentence_count', 'exclamation_count', 'question_count', 'comma_count', 'uppercase_ratio', 'number_count', 'special_char_count']
Combined features shape - Train: (6193, 15010)  Test: (2065, 15010)


In [ ]:
base_models = [
    ('svm', LinearSVC(C=1, loss='squared_hinge', random_state=42, max_iter=1500, dual=False)),
    ('lr', LogisticRegression(C=1, max_iter=1000, random_state=42)),
    ('rf', RandomForestClassifier(n_estimators=302, random_state=42, n_jobs=-1))
]

meta_learner = LinearSVC(C=1.5, loss='squared_hinge', random_state=42, max_iter=1000, dual=False)

# Stacking classifier
stacking = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_learner,
    cv=6
)

stacking.fit(X_train_combined, y_train)
y_pred_stack = stacking.predict(X_test_combined)
acc_stack = accuracy_score(y_test, y_pred_stack)
print(f"Stacking Ensemble: {acc_stack:.4f}")

Stacking Ensemble: 0.8804


In [40]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

print("="*50)

# Các mô hình cơ sở
base_models = [
    ('svm', LinearSVC(C=1, loss='squared_hinge', random_state=42, max_iter=1500, dual=False)),
    ('lr', LogisticRegression(C=1, max_iter=1000, random_state=42)),
    ('rf', RandomForestClassifier(n_estimators=302, random_state=42, n_jobs=-1))
]

meta_learner = LinearSVC(C=1.5, loss='squared_hinge', random_state=42, max_iter=1000, dual=False)

# Train và in kết quả từng mô hình cơ sở
for name, model in base_models:
    model.fit(X_train_combined, y_train)
    y_pred = model.predict(X_test_combined)
    acc = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {acc:.4f}")

print("-"*50)

# Stacking classifier
stacking = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_learner,
    cv=6
)

stacking.fit(X_train_combined, y_train)
y_pred_stack = stacking.predict(X_test_combined)
acc_stack = accuracy_score(y_test, y_pred_stack)
print(f"Stacking Ensemble Accuracy: {acc_stack:.4f}")


svm Accuracy: 0.8673
lr Accuracy: 0.8479
rf Accuracy: 0.8305
--------------------------------------------------
Stacking Ensemble Accuracy: 0.8804
